### TODO
- Now we can take the special cases we added to info_grupos back to frac_vars.xlsx since we have them in cols to avoid.
- The HelperFunctions can raise Errors or warning that help us identify null values, mismatching vars and things like that.

In [28]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
import pickle
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
from info_grupos import empirial_vars_to_avoid, frac_vars_special_cases_list
from genera_muestra import GenerateLHCSample
from utils import HelperFunctions

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

In [29]:
# Record the start time
start_time = time.time()

In [30]:
## Define paths and global variables

FILE_PATH = os.getcwd()
build_path = lambda PATH : os.path.abspath(os.path.join(*PATH))

DATA_PATH = build_path([FILE_PATH, "..", "data"])
OUTPUT_PATH = build_path([FILE_PATH, "..", "output"])

SSP_OUTPUT_PATH = build_path([OUTPUT_PATH, "ssp"])

REAL_DATA_FILE_PATH = build_path([DATA_PATH, "real_data.csv"]) 

SALIDAS_EXPERIMENTOS_PATH = build_path([OUTPUT_PATH, "experiments"]) 

INPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_inputs"])
OUTPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_outputs"])

target_country = "croatia"
helper_functions = HelperFunctions()

In [31]:
### Cargamos datos de ejemplo de costa rica

examples = SISEPUEDEExamples()
cr = examples("input_data_frame")

In [32]:
df_input = pd.read_csv(REAL_DATA_FILE_PATH)
df_input.head()

,region,iso_code3,period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,croatia,HRV,0,8807000,70,2923,31.751466,6468,315,508,...,5.546667,28.8742,0.602367,1.930675,0.5205,2.638183,0,0,35.76480,21.067738
1,croatia,HRV,1,8807000,70,2923,31.751466,6468,315,508,...,5.555383,29.6558,0.479900,2.304050,0.7342,3.256933,0,0,47.57660,20.412554
2,croatia,HRV,2,8807000,70,2923,31.751466,6468,315,508,...,4.304906,30.0390,0.360400,2.075950,0.8615,3.199083,0,0,41.09780,22.329531
3,croatia,HRV,3,8807000,70,2923,31.751466,6468,315,508,...,6.272229,30.0390,0.160300,2.662133,0.9852,3.266680,0,0,37.42445,21.505800
4,croatia,HRV,4,8807000,70,2923,31.751466,6468,315,508,...,5.878853,30.0390,0.196325,2.543567,0.9045,4.068040,0,0,39.81490,21.874247


In [33]:
df_input = df_input.rename(columns={'period':'time_period'})
df_input = helper_functions.add_missing_cols(cr, df_input.copy())
df_input = df_input.drop(columns='iso_code3')

In [34]:
df_input.head()

,region,time_period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,croatia,0,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,croatia,1,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,croatia,2,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,croatia,3,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,croatia,4,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [35]:
# Double checking that our df is in the correct shape MAKE SURE THIS IS OK THEY HAVE TO BE EQUAL!
compare_dfs(cr, df_input)

Columns in df1 but not in df2: set()
Columns in df2 but not in df1: set()


In [36]:
# Checking if there are any columns with null values in it
columns_with_na = df_input.columns[df_input.isna().any()].tolist()

print(columns_with_na)

[]


In [37]:
columns_all_999 = df_input.columns[(df_input == -999).any()].tolist()
columns_all_999

['frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_hydropower',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_nuclear',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_ocean',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_oil',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_solar',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_waste_incineration',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_wind',
 'limit_gnrl_annual_emissions_mt_ch

In [38]:
# Avoid land use stuff and some frac special cases
pij_cols = [col for col in df_input.columns if col.startswith('pij')]
cols_to_avoid = pij_cols + frac_vars_special_cases_list + columns_all_999 + empirial_vars_to_avoid
campos_estresar = helper_functions.get_indicators_col_names(df_input, cols_with_issue=cols_to_avoid)

Expected length after removal: 2034
Actual length of col_names: 2034
All columns in cols_to_avoid were successfully removed.


In [39]:
generate_sample = GenerateLHCSample(n=100, n_var=len(campos_estresar))
generate_sample.generate_sample()
with open('sample_scaled.pickle', 'rb') as handle:
    sample_scaled = pickle.load(handle)

In [40]:
id_experimento = 1
    
df_estresado = df_input.copy()
df_estresado[campos_estresar]  = (df_input[campos_estresar]*sample_scaled[id_experimento]).to_numpy()

In [41]:
# Load new groups that need normalization
df_frac_vars = pd.read_excel('frac_vars.xlsx', sheet_name='frac_vars_no_special_cases')
df_frac_vars.head()

,frac_var_name,frac_var_name_prefix
0,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1
1,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl1
2,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2
3,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_bevs_and_spices_cl2
4,frac_agrc_cereals_cl1_temperate,frac_agrc_cereals_cl1


In [42]:
need_norm_prefix = df_frac_vars.frac_var_name_prefix.unique()

In [43]:
### Normaliza grupo de variables para que sumen 1 en conjunto

for grupo in need_norm_prefix:
    vars_grupo = [i for i in df_estresado.columns if grupo in i]
    
    # Skip normalization for columns in cols_to_avoid
    if any(col in cols_to_avoid for col in vars_grupo):
        continue

    # Apply conditional log transformation
    df_estresado[vars_grupo] = df_estresado[vars_grupo].applymap(lambda y: -np.log(y) if y != 0 else 0)
    
    # Check if the sum is zero before normalizing
    sum_values = df_estresado[vars_grupo].sum(axis=1)
    df_estresado[vars_grupo] = df_estresado[vars_grupo].div(sum_values, axis=0).fillna(0)


# This is also an special case
ce_problematic = ['frac_waso_biogas_food',
                  'frac_waso_biogas_sludge',
                  'frac_waso_biogas_yard',
                  'frac_waso_compost_food',
                  'frac_waso_compost_methane_flared',
                  'frac_waso_compost_sludge',
                  'frac_waso_compost_yard']

# Apply conditional log transformation
df_estresado[ce_problematic] = df_estresado[ce_problematic].applymap(lambda y: -np.log(y) if y != 0 else 0)
# Check if the sum is zero before normalizing
sum_values = df_estresado[ce_problematic].sum(axis=1)
df_estresado[ce_problematic] = df_estresado[ce_problematic].div(sum_values, axis=0).fillna(0)



####################

In [44]:
# # Assuming df_estresado is defined and contains columns
# vars_grupo = [i for i in df_estresado.columns if i.startswith('frac_')]

# df_frac_vars = pd.DataFrame(vars_grupo, columns=['frac_var_name'])
# df_frac_vars.sort_values(by='frac_var_name', inplace=True)

# # Extract prefix by removing the last '_{word}' segment
# df_frac_vars['frac_var_name_prefix'] = df_frac_vars['frac_var_name'].apply(lambda x: '_'.join(x.split('_')[:-1]))

# df_frac_vars.to_csv('frac_vars.csv', index=False)

###########

In [45]:
# Checking if there are any columns with null values in it
columns_with_na = df_estresado.columns[df_estresado.isna().any()].tolist()
print(columns_with_na)
if columns_with_na:
    df_estresado[columns_with_na] = df_estresado[columns_with_na].fillna(0)

columns_with_na = df_estresado.columns[df_estresado.isna().any()].tolist()
print(columns_with_na)

[]
[]


In [46]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_estresado,
)

##  SETUP SOME SISEPUEDE STUFF

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

# set an ouput path and instantiate

trf.instantiate_default_strategy_directory(
        transformers,
        SSP_OUTPUT_PATH,
    )

# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
        SSP_OUTPUT_PATH,
        transformers = transformers,
    )

strategies = trf.Strategies(
        transformations,
        export_path = "transformations",
        prebuild = True,
    )

In [47]:


# call the example
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
        df_trajgroup = df_vargroups,
        include_simplex_group_as_trajgroup = True,
        strategies = [0, 1000],
    )



0

In [48]:
import sisepuede as si
ssp = si.SISEPUEDE(
        "calibrated",
        initialize_as_dummy = False, # no connection to Julia is initialized if set to True
        regions = [target_country],
        db_type = "csv",
        strategies = strategies,
        try_exogenous_xl_types_in_variable_specification = True,
    )

2024-11-13 13:39:35,356 - INFO - Successfully initialized SISEPUEDEFileStructure.
2024-11-13 13:39:35,356 - INFO - Successfully initialized SISEPUEDEFileStructure.
2024-11-13 13:39:35,361 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2024-11-13 13:39:35,361 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2024-11-13 13:39:35,362 - INFO - 	Setting export engine to 'csv'.
2024-11-13 13:39:35,362 - INFO - 	Setting export engine to 'csv'.
2024-11-13 13:39:35,363 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2024-11-13 13:39:35,363 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2024-11-13 13:39:35,365 - INFO - Successfully instantiated table ANALYSIS_METADATA
2024-11-13 13:39:35,365 - INFO - Successfully

In [49]:
# Checks if the land use reallocation factor is set to 0.0
helper_functions.check_land_use_factor(ssp_object=ssp, target_country=target_country)

In [50]:
# Create parameters dict for the model to run
dict_run = {
        ssp.key_future: [0],
        ssp.key_design: [0],
        ssp.key_strategy: [
            0,
            1000,
        ],
    }


In [51]:
# we'll save inputs since we're doing a small set of runs
ssp.project_scenarios(
        dict_run,
        save_inputs = True,
    )

2024-11-13 13:40:12,055 - INFO - 
***	STARTING REGION croatia	***

2024-11-13 13:40:12,055 - INFO - 
***	STARTING REGION croatia	***

2024-11-13 13:40:15,872 - INFO - Trying run primary_id = 0 in region croatia
2024-11-13 13:40:15,872 - INFO - Trying run primary_id = 0 in region croatia
2024-11-13 13:40:15,873 - INFO - Running AFOLU model
2024-11-13 13:40:15,873 - INFO - Running AFOLU model
2024-11-13 13:40:16,075 - INFO - AFOLU model run successfully completed
2024-11-13 13:40:16,075 - INFO - AFOLU model run successfully completed
2024-11-13 13:40:16,077 - INFO - Running CircularEconomy model
2024-11-13 13:40:16,077 - INFO - Running CircularEconomy model
2024-11-13 13:40:16,138 - INFO - CircularEconomy model run successfully completed
2024-11-13 13:40:16,138 - INFO - CircularEconomy model run successfully completed
2024-11-13 13:40:16,140 - INFO - Running IPPU model
2024-11-13 13:40:16,140 - INFO - Running IPPU model
2024-11-13 13:40:16,228 - INFO - IPPU model run successfully complet

2024-13-Nov 13:40:16.425 Opened SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-13-Nov 13:40:16.480 Added NEMO structure to SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-13-Nov 13:40:18.786 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.


┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112


2024-13-Nov 13:43:16.152 Finished modeling scenario.


2024-11-13 13:43:16,409 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-13 13:43:16,409 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-13 13:43:16,456 - INFO - EnergyProduction model run successfully completed
2024-11-13 13:43:16,456 - INFO - EnergyProduction model run successfully completed
2024-11-13 13:43:16,458 - INFO - Running Energy (Fugitive Emissions)
2024-11-13 13:43:16,458 - INFO - Running Energy (Fugitive Emissions)
2024-11-13 13:43:16,514 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-13 13:43:16,514 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-13 13:43:16,515 - INFO - Appending Socioeconomic outputs
2024-11-13 13:43:16,515 - INFO - Appending Socioeconomic outputs
2024-11-13 13:43:16,526 - INFO - Socioeconomic outputs successfully appended.
2024-11-13 13:43:16,526 - INFO - Socioeconomic outputs successfully appended.
2024-11-13 13:43:16,532 - INFO

2024-13-Nov 13:43:19.850 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.


┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112


2024-13-Nov 13:45:57.395 Finished modeling scenario.


2024-11-13 13:45:57,606 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-13 13:45:57,606 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-13 13:45:57,624 - INFO - EnergyProduction model run successfully completed
2024-11-13 13:45:57,624 - INFO - EnergyProduction model run successfully completed
2024-11-13 13:45:57,625 - INFO - Running Energy (Fugitive Emissions)
2024-11-13 13:45:57,625 - INFO - Running Energy (Fugitive Emissions)
2024-11-13 13:45:57,682 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-13 13:45:57,682 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-13 13:45:57,683 - INFO - Appending Socioeconomic outputs
2024-11-13 13:45:57,683 - INFO - Appending Socioeconomic outputs
2024-11-13 13:45:57,697 - INFO - Socioeconomic outputs successfully appended.
2024-11-13 13:45:57,697 - INFO - Socioeconomic outputs successfully appended.
2024-11-13 13:45:57,704 - INFO

{'croatia': [0, 1001]}

In [52]:
INPUTS_ESTRESADOS_FILE_PATH = build_path([INPUTS_ESTRESADOS_PATH, f"sim_input_{id_experimento}.csv"])
OUTPUTS_ESTRESADOS_FILE_PATH = build_path([OUTPUTS_ESTRESADOS_PATH, f"sim_output_{id_experimento}.csv"])


df_out = ssp.read_output(None)
df_out.to_csv(OUTPUTS_ESTRESADOS_FILE_PATH, index=False)
df_estresado[campos_estresar].to_csv(INPUTS_ESTRESADOS_FILE_PATH, index=False)

helper_functions.print_elapsed_time(start_time)

------------------------ EXECUTION TIME: 399.9020211696625 seconds ------------------------


In [ ]:
df_out.to_csv('croatia_dummy_output.csv', index=False)

: 